---

## 1. O Conceito: Managed vs. External Tables

A aula foca em como o Databricks gerencia os arquivos no armazenamento (DBFS/S3/ADLS).

* **Managed Table (Tabela Gerenciada):** Se você der um `DROP TABLE`, o Databricks apaga os **metadados** (o registro na interface) e os **dados reais** (os arquivos).
* **External/Unmanaged Table:** Se você der um `DROP TABLE`, o Databricks apaga apenas os metadados. Os arquivos originais continuam lá.

---

## 2. Exemplos Práticos para o Databricks Free

Copie e cole estes blocos em células diferentes do seu notebook.

### Passo A: Criar Bancos de Dados

Em vez de usar `${da.db_name}`, vamos usar um nome fixo como `meu_teste`.

```sql
-- Criando banco no local padrão
CREATE DATABASE IF NOT EXISTS teste_default_location;

-- Criando banco em um local específico (Custom)
-- No Community Edition, usamos o diretório /FileStore/
CREATE DATABASE IF NOT EXISTS teste_custom_location
LOCATION '/FileStore/tables/_custom_location.db';

```

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS teste_default_location;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS hive_metastore.teste_custom_location
LOCATION 'dbfs:/FileStore/_custom_location.db';

In [0]:
%sql
SHOW CATALOGS;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS meu_schema;

In [0]:
%sql
DESCRIBE SCHEMA EXTENDED meu_schema;

### Passo B: Criar Tabela Gerenciada (Managed)

Aqui você verá que, ao não especificar um `LOCATION`, o Spark coloca os dados dentro da pasta do banco de dados.

```sql
USE teste_default_location;

CREATE OR REPLACE TABLE tabela_gerenciada (
  id INT,
  nome STRING
);

INSERT INTO tabela_gerenciada VALUES (1, 'Exemplo 1'), (2, 'Exemplo 2');

-- Comando para ver onde os arquivos foram parar
DESCRIBE DETAIL tabela_gerenciada;

```

### Passo C: Criar Tabela Externa (External)

Para isso, primeiro precisamos de arquivos. Vamos simular isso criando uma VIEW temporária e salvando em um local fixo.

```sql
-- Criando um CSV fictício via código para teste
-- (Substituindo a parte do 'temp_delays' da sua aula)
CREATE OR REPLACE TEMPORARY VIEW dados_ficticios AS
SELECT 100 AS voo, 'SAO' AS origem, 'NYC' AS destino;

-- Criando a tabela EXTERNA apontando para um caminho específico
CREATE OR REPLACE TABLE tabela_externa 
LOCATION '/FileStore/tabelas/meus_dados_externos' 
AS SELECT * FROM dados_ficticios;

-- Verifique o campo 'Type' (será EXTERNAL)
DESCRIBE FORMATTED tabela_externa;

```

---


## 3. O Teste de "Fogo" (O que a aula quer mostrar)

Para entender a diferença crucial, execute estes comandos:

1. **Teste na Gerenciada:**
```sql
DROP TABLE teste_default_location.tabela_gerenciada;
-- Se você tentar listar os arquivos via Python agora, a pasta sumiu.

```


2. **Teste na Externa:**
```sql
DROP TABLE tabela_externa;
-- O comando OK aparece, a tabela some da interface lateral.
-- MAS, execute o código abaixo (Python) e verá que os dados AINDA ESTÃO LÁ:

```


```python
# Verificando que os arquivos permanecem
display(dbutils.fs.ls('/FileStore/tabelas/meus_dados_externos'))

```



---

## Resumo dos Comandos Úteis

Como você não tem o script `DA`, use estes substitutos:

* Para ver arquivos: `%fs ls /caminho/` ou `dbutils.fs.ls("/caminho/")`
* Para ver detalhes da tabela: `DESCRIBE DETAIL nome_tabela`
* Para ver o DDL de criação: `SHOW CREATE TABLE nome_tabela`